In [1]:
import os

In [2]:
%pwd

'/home/ubuntu/learning/mlops/pacmann/lazada-id-reviews/notebooks'

In [3]:
# Change to the main directory
# So, it's executed from main directory
os.chdir("../")

In [4]:
with open('.env') as f:
    os.environ.update(
        line.strip().split('=') for line in f
)

In [5]:
%pwd

'/home/ubuntu/learning/mlops/pacmann/lazada-id-reviews'

### Unit Testing Config

This code will be apply in `src/LadazaIDReview/entity/config_entity.py`.

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class UnitTestConfig:
    root_dir: Path
    mlflow_tracking_uri: str
    mlflow_model_name: str
    mlflow_deploy_model_alias: str
    mlflow_input_example_path: Path
    app_endpoint: str
    

### Unit Testing Config Manager

This code will be apply in `src/LazadaIDReview/config/configurations.py`.

In [7]:
from LazadaIDReviews.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from LazadaIDReviews.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_unit_test_config(self) -> UnitTestConfig:
        """read training evaluation config file and store as 
        config entity then apply the dataclasses
        
        Returns:
            config: UnitTestConfig type
        """
        predict_config = self.config.predict
        unit_test_config = self.config.unit_test

        create_directories([unit_test_config.root_dir])

        config = UnitTestConfig(
            root_dir=unit_test_config.root_dir,
            mlflow_tracking_uri=os.environ["MLFLOW_TRACKING_URI"],
            mlflow_model_name=predict_config.mlflow_model_name,
            mlflow_deploy_model_alias=os.environ["MLFLOW_DEPLOY_MODEL_ALIAS"],
            mlflow_input_example_path=unit_test_config.mlflow_input_example_path,
            app_endpoint=os.environ["APP_ENDPOINT"]
        )

        return config

In [9]:
from mlflow.artifacts import download_artifacts
from mlflow import MlflowClient
from mlflow import pyfunc

---

**Debug**: Explain when doing the preparation test in the notebook with MLflow like load input example and etc.

In [10]:
config = ConfigurationManager()
unit_test_config = config.get_unit_test_config()

[2024-09-11 08:31:38,789: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-11 08:31:38,792: INFO: common: yaml file: metrics/params.yaml loaded successfully]
[2024-09-11 08:31:38,792: INFO: common: created directory at: artifacts]
[2024-09-11 08:31:38,794: INFO: common: created directory at: artifacts/test]


Select the deployed model from MLflow.

In [11]:
client = MlflowClient(tracking_uri=unit_test_config.mlflow_tracking_uri)
selected_model = client.get_model_version_by_alias(
    unit_test_config.mlflow_model_name, 
    unit_test_config.mlflow_deploy_model_alias
)

selected_model.source

'mlflow-artifacts:/1/1f046f7844764a9a907682ce09cca37e/artifacts/models'

In [12]:
loaded_model = pyfunc.load_model(model_uri=selected_model.source)
loaded_model

/home/ubuntu/learning/mlops/pacmann/lazada-id-reviews/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


mlflow.pyfunc.loaded_model:
  artifact_path: models
  flavor: mlflow.sklearn
  run_id: 1f046f7844764a9a907682ce09cca37e

Get the model `run_id`.

In [13]:
selected_run_id = selected_model.run_id
selected_run_id

'1f046f7844764a9a907682ce09cca37e'

In [14]:
download_artifacts(
    run_id=selected_run_id,
    artifact_path=unit_test_config.mlflow_input_example_path,
    dst_path=unit_test_config.root_dir
)

'/home/ubuntu/learning/mlops/pacmann/lazada-id-reviews/artifacts/test/models/input_example.json'

In [15]:
import json

f = open(f"{unit_test_config.root_dir}/{unit_test_config.mlflow_input_example_path}")
input_example = json.load(f)
input_example

{'columns': ['reviewContents'],
 'data': [['brg mulus,kiriman jg cepat, thx'],
  ['Terima kasih Adata Store dan Lazada, barang sudah sampai, mantab'],
  ['Trimaksih Lazada barang nya sangat bagus dan memuas kan ..  Waktu pesen kemarin blm ada free besi ny .  Skrng udh ada .  Heee heee ..'],
  ['barang sampai juga agak lama pengirimanya,, Alhamdulillah barang gak ada yg cacat,,'],
  ['bagus, dapet bonus kabel strap lumayan 😁'],
  ['Mohon tulis harga dengan benar'],
  ['mantab jiwa,,sory barang ga sempat dipoto'],
  ['Recomended banget lur...'],
  ['Terima kasih Lazada pengiriman cepat dan packingnya rapi, barang sesuai dgn pesanan saya...tp belum saya coba aktifkan'],
  ['Saya kira karena liburan bakal lama sampainya, ternyata tidak,  Barang asli setelah dibuka,sama dengan foto   Thanks lazada, memang effortless banget']]}

We use the input data from MLflow input examples and try to match with the MLflow input example format.

In [16]:
request_body = {
    input_example["columns"][0]: [i[0] for i in input_example['data']]
}

request_body

{'reviewContents': ['brg mulus,kiriman jg cepat, thx',
  'Terima kasih Adata Store dan Lazada, barang sudah sampai, mantab',
  'Trimaksih Lazada barang nya sangat bagus dan memuas kan ..  Waktu pesen kemarin blm ada free besi ny .  Skrng udh ada .  Heee heee ..',
  'barang sampai juga agak lama pengirimanya,, Alhamdulillah barang gak ada yg cacat,,',
  'bagus, dapet bonus kabel strap lumayan 😁',
  'Mohon tulis harga dengan benar',
  'mantab jiwa,,sory barang ga sempat dipoto',
  'Recomended banget lur...',
  'Terima kasih Lazada pengiriman cepat dan packingnya rapi, barang sesuai dgn pesanan saya...tp belum saya coba aktifkan',
  'Saya kira karena liburan bakal lama sampainya, ternyata tidak,  Barang asli setelah dibuka,sama dengan foto   Thanks lazada, memang effortless banget']}

Test the `app.py` with http request with MLflow input data example.

In [17]:
import requests

result = requests.post(url=unit_test_config.app_endpoint, json=request_body)
y_predict = result.json()

In [18]:
y_predict

[5, 5, 5, 4, 3, 1, 5, 5, 5, 3]

---

### Unit Testing

This code in `src/LazadaIDReview/components/unit_testing.py`.

In [19]:
import json
import requests

from LazadaIDReviews import logger

class UnitTesting:
    def __init__(self, config: UnitTestConfig):
        self.config = config
        self.req_body_key = None
        self.req_body = None
    
    def set_request_body(self) -> None:
        """predict the data with linear regression model

        Raises:
            client_error: error when access mlflow to get deployed model
            download_error: error when download vectorizer from mlflow artifact
        """
        try:
            logger.info("Set MLflow Client.")
            client = MlflowClient(tracking_uri=self.config.mlflow_tracking_uri)
            selected_model = client.get_model_version_by_alias(
                self.config.mlflow_model_name, 
                self.config.mlflow_deploy_model_alias
            )
            
            logger.info("Get the deployed model run id.")
            selected_run_id = selected_model.run_id
        except Exception as client_error:
            logger.error(client_error)
            raise client_error

        try:
            logger.info("Downloading vectorizer from MLflow's artifacts.")
            download_artifacts(
                run_id=selected_run_id,
                artifact_path = self.config.mlflow_input_example_path,
                dst_path = self.config.root_dir
            )
        except Exception as download_error:
            logger.error(download_error)
            raise download_error
        
        logger.info("Open MLflow input example.")
        f = open(f"{self.config.root_dir}/{self.config.mlflow_input_example_path}")
        input_example = json.load(f)

        # handle mlflow input example data
        data_key = input_example["columns"][0]
        data_val = [i[0] for i in input_example['data']]

        # request params
        self.req_body_key = data_key
        self.req_body = {
            data_key: data_val
        }
        
    def get_request_body_value(self) -> list:
        """get the request body data

        Returns:
            req_body: list type
        """
        logger.info("Get MLflow input example value.")
        req_body_value = self.req_body[self.req_body_key]
        return req_body_value
    
    def get_output_length(self):
        """get the output length of the predict result

        Returns:
            len_result: list type
        """
        logger.info("Get predicted result length.")
        result = requests.post(
            url = self.config.app_endpoint, 
            json = self.req_body
        )
        len_result = len(result.json())
        return len_result

    def is_output_type_list(self) -> bool:
        """check if the output file is list data type

        Returns:
            is_list: bool type
        """
        logger.info("Check is the predicted output is list.")
        result = requests.post(
            url=self.config.app_endpoint, 
            json=self.req_body
        )
        is_list = type(result.json()) is list
        return is_list

    def is_output_type_consistent(self) -> bool:
        """check if the output file have consistent
        data type inside a list

        Returns:
            bool type
        """
        logger.info("Check is each predicted output is integer")
        result = requests.post(
            url=self.config.app_endpoint, 
            json=self.req_body
        )
        for result in result.json():
            if type(result) is not int:
                return False
        return True

### Run Testing

**Debug**: Simulate the unit testing without library.

In [20]:
try:
    config = ConfigurationManager()
    unit_testing_config = config.get_unit_test_config()
    unit_test = UnitTesting(config=unit_testing_config)
    unit_test.set_request_body()
    
    print("Review Contents: ")
    for content in unit_test.get_request_body_value():
        print(content)
    
    print("\nBegin tests:")
    print(f"Is same size: {unit_test.get_output_length() == len(unit_test.get_request_body_value())}")
    print(f"Is the output is list: {unit_test.is_output_type_list() == True}")
    print(f"Is the output consistent: {unit_test.is_output_type_consistent() == True}")
except Exception as e:
    logger.error(e)
    raise e

[2024-09-11 08:31:39,720: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-11 08:31:39,722: INFO: common: yaml file: metrics/params.yaml loaded successfully]
[2024-09-11 08:31:39,723: INFO: common: created directory at: artifacts]
[2024-09-11 08:31:39,723: INFO: common: created directory at: artifacts/test]
[2024-09-11 08:31:39,724: INFO: 999233882: Set MLflow Client.]
[2024-09-11 08:31:39,733: INFO: 999233882: Get the deployed model run id.]
[2024-09-11 08:31:39,733: INFO: 999233882: Downloading vectorizer from MLflow's artifacts.]


[2024-09-11 08:31:39,767: INFO: 999233882: Open MLflow input example.]
Review Contents: 
[2024-09-11 08:31:39,768: INFO: 999233882: Get MLflow input example value.]
brg mulus,kiriman jg cepat, thx
Terima kasih Adata Store dan Lazada, barang sudah sampai, mantab
Trimaksih Lazada barang nya sangat bagus dan memuas kan ..  Waktu pesen kemarin blm ada free besi ny .  Skrng udh ada .  Heee heee ..
barang sampai juga agak lama pengirimanya,, Alhamdulillah barang gak ada yg cacat,,
bagus, dapet bonus kabel strap lumayan 😁
Mohon tulis harga dengan benar
mantab jiwa,,sory barang ga sempat dipoto
Recomended banget lur...
Terima kasih Lazada pengiriman cepat dan packingnya rapi, barang sesuai dgn pesanan saya...tp belum saya coba aktifkan
Saya kira karena liburan bakal lama sampainya, ternyata tidak,  Barang asli setelah dibuka,sama dengan foto   Thanks lazada, memang effortless banget

Begin tests:
[2024-09-11 08:31:39,768: INFO: 999233882: Get predicted result length.]
[2024-09-11 08:31:39,938:

Is the output is list: True
[2024-09-11 08:31:40,141: INFO: 999233882: Check is each predicted output is integer]
Is the output consistent: True
